# **1. Importaciones**

In [20]:
# Conexion de google colab con drive:
from google.colab import drive

# Importar librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           confusion_matrix, classification_report, roc_auc_score, roc_curve)


import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from scipy.stats import randint, uniform
import time
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import brier_score_loss


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Extracción del Archivo Tratado**

In [22]:
# Cargar dataset sucio
df = pd.read_csv('/content/drive/MyDrive/ONE/HACKATHON/DATASET/dataset_SUCIO/dataset_sucio_final.csv')

print(f" Shape: {df.shape}")

 Shape: (5000, 10)


In [23]:
df.head()

,antiguedad,plan,metodo_pago,facturas_impagas,frecuencia_uso,tickets_soporte,tipo_contrato,cambios_plan,canal_adquisicion,churn
0,16,estandar,transferencia_bancaria,2.0,NaN,0.0,mensual,3,call_center,0
1,120,estandar,efectivo,3.0,2.0,10.0,mensual,2,referido,1
2,29,premium,tarjeta_credito,NaN,28.0,1.0,anual,3,web,0
3,48,estandar,tarjeta_debito,0.0,9.0,2.0,mensual,1,referido,0
4,13,estandar,tarjeta_credito,1.0,19.0,5.0,mensual,0,web,0


# **3. Información General del Dataset**

In [24]:
print("INFORMACIÓN GENERAL DEL DATASET")
df.info()

INFORMACIÓN GENERAL DEL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   antiguedad         5000 non-null   int64  
 1   plan               5000 non-null   object 
 2   metodo_pago        4507 non-null   object 
 3   facturas_impagas   4518 non-null   float64
 4   frecuencia_uso     4519 non-null   float64
 5   tickets_soporte    4517 non-null   float64
 6   tipo_contrato      5000 non-null   object 
 7   cambios_plan       5000 non-null   int64  
 8   canal_adquisicion  4500 non-null   object 
 9   churn              5000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 390.8+ KB


# **4. Limpieza de Datos**

## **4.1 Verificando Duplicados**

In [25]:
# 1° Crear copia para no modificar el original
df_limpio = df.copy()


print("\n VERIFICANDO DUPLICADOS...")
duplicados = df_limpio.duplicated().sum()
print(f"   Registros duplicados: {duplicados}")

if duplicados > 0:
    df_limpio = df_limpio.drop_duplicates()
    print(f"   {duplicados} duplicados eliminados")
else:
    print("   No hay duplicados")


 VERIFICANDO DUPLICADOS...
   Registros duplicados: 0
   No hay duplicados


## **4.2 Verificando Valores Nulos**

In [26]:
print("\n TRATAMIENTO DE VALORES NULOS...")
print(f"\n   Nulos antes de limpieza:")
print(df_limpio.isnull().sum())

# Estrategia de imputación
# Numéricas: mediana
# Categóricas: moda

df_limpio['facturas_impagas'] = df_limpio['facturas_impagas'].fillna(df_limpio['facturas_impagas'].median())
df_limpio['frecuencia_uso'] = df_limpio['frecuencia_uso'].fillna(df_limpio['frecuencia_uso'].median())
df_limpio['tickets_soporte'] = df_limpio['tickets_soporte'].fillna(df_limpio['tickets_soporte'].median())

df_limpio['metodo_pago'] = df_limpio['metodo_pago'].fillna(df_limpio['metodo_pago'].mode()[0])
df_limpio['canal_adquisicion'] = df_limpio['canal_adquisicion'].fillna(df_limpio['canal_adquisicion'].mode()[0])

print(f"\n   Nulos después de limpieza:")
print(df_limpio.isnull().sum())



 TRATAMIENTO DE VALORES NULOS...

   Nulos antes de limpieza:
antiguedad             0
plan                   0
metodo_pago          493
facturas_impagas     482
frecuencia_uso       481
tickets_soporte      483
tipo_contrato          0
cambios_plan           0
canal_adquisicion    500
churn                  0
dtype: int64

   Nulos después de limpieza:
antiguedad           0
plan                 0
metodo_pago          0
facturas_impagas     0
frecuencia_uso       0
tickets_soporte      0
tipo_contrato        0
cambios_plan         0
canal_adquisicion    0
churn                0
dtype: int64


## **4.3 Correccón de Tipo de Datos**

In [27]:

print("\nCORRIGIENDO TIPOS DE DATOS...")

# Convertir float a int (donde corresponda)
df_limpio['facturas_impagas'] = df_limpio['facturas_impagas'].astype(int)
df_limpio['frecuencia_uso'] = df_limpio['frecuencia_uso'].astype(int)
df_limpio['tickets_soporte'] = df_limpio['tickets_soporte'].astype(int)


CORRIGIENDO TIPOS DE DATOS...


## **4.4 Normalizar categóricos mal formateados**


In [28]:
print("\n NORMALIZANDO VALORES CATEGÓRICOS...")

# Normalizar 'plan'
df_limpio['plan'] = df_limpio['plan'].str.lower().str.strip()
df_limpio['plan'] = df_limpio['plan'].replace({
    'básico': 'basico',
    'estandar': 'estandar',
    'estándar': 'estandar',
    'standard': 'estandar',
    'premium': 'premium',
    'vip': 'premium'
})

# Normalizar 'metodo_pago'
df_limpio['metodo_pago'] = df_limpio['metodo_pago'].str.lower().str.strip().str.replace(' ', '_')
df_limpio['metodo_pago'] = df_limpio['metodo_pago'].replace({
    'tarjeta_credito': 'tarjeta_credito',
    'tarjeta_de_credito': 'tarjeta_credito',
    'tarjeta_crédito': 'tarjeta_credito',
    'tarjeta_debito': 'tarjeta_debito',
    'tarjeta_de_debito': 'tarjeta_debito',
    'tarjeta_débito': 'tarjeta_debito',
    'transferencia_bancaria': 'transferencia_bancaria',
    'paypal': 'transferencia_bancaria',
    'transferencia': 'transferencia_bancaria',
    'efectivo': 'efectivo'
})

# Normalizar 'tipo_contrato'
df_limpio['tipo_contrato'] = df_limpio['tipo_contrato'].str.lower().str.strip()

# Normalizar 'canal_adquisicion'
df_limpio['canal_adquisicion'] = df_limpio['canal_adquisicion'].str.lower().str.strip().str.replace(' ', '_')


 NORMALIZANDO VALORES CATEGÓRICOS...


## **4.5 Eliminar y corregir Outliers**

In [29]:

print("\n TRATANDO OUTLIERS...")

# Antiguedad: valores negativos o extremadamente altos
df_limpio = df_limpio[df_limpio['antiguedad'] >= 0]
df_limpio = df_limpio[df_limpio['antiguedad'] <= 120]
print(f"   - antiguedad: valores fuera de rango [0-120] eliminados")

# Frecuencia_uso: valores negativos o imposibles
df_limpio = df_limpio[df_limpio['frecuencia_uso'] >= 0]
df_limpio = df_limpio[df_limpio['frecuencia_uso'] <= 30]
print(f"   - frecuencia_uso: valores fuera de rango [0-30] eliminados")

# Facturas_impagas: valores negativos
df_limpio = df_limpio[df_limpio['facturas_impagas'] >= 0]
df_limpio = df_limpio[df_limpio['facturas_impagas'] <= 10]
print(f"   - facturas_impagas: valores fuera de rango [0-10] eliminados")

# Tickets_soporte: valores extremos
df_limpio = df_limpio[df_limpio['tickets_soporte'] >= 0]
df_limpio = df_limpio[df_limpio['tickets_soporte'] <= 50]
print(f"   - tickets_soporte: valores fuera de rango [0-50] eliminados")

# Cambios_plan: valores negativos
df_limpio = df_limpio[df_limpio['cambios_plan'] >= 0]
df_limpio = df_limpio[df_limpio['cambios_plan'] <= 5]
print(f"   - cambios_plan: valores fuera de rango [0-5] eliminados")


 TRATANDO OUTLIERS...
   - antiguedad: valores fuera de rango [0-120] eliminados
   - frecuencia_uso: valores fuera de rango [0-30] eliminados
   - facturas_impagas: valores fuera de rango [0-10] eliminados
   - tickets_soporte: valores fuera de rango [0-50] eliminados
   - cambios_plan: valores fuera de rango [0-5] eliminados


## **4.6 Resetear Índices**

In [30]:
df_limpio = df_limpio.reset_index(drop=True)

## **4.7 Resumen Limpieza Datos**

In [31]:

print(" RESUMEN DE LIMPIEZA")

print(f"\n Registros originales: {len(df)}")
print(f"    Registros después de limpieza: {len(df_limpio)}")
print(f"    Registros eliminados: {len(df) - len(df_limpio)} ({(len(df) - len(df_limpio))/len(df)*100:.2f}%)")
print(f"    Nulos restantes: {df_limpio.isnull().sum().sum()}")
print(f"\n    Tipos de datos:")
print(df_limpio.dtypes)

 RESUMEN DE LIMPIEZA

 Registros originales: 5000
    Registros después de limpieza: 4251
    Registros eliminados: 749 (14.98%)
    Nulos restantes: 0

    Tipos de datos:
antiguedad            int64
plan                 object
metodo_pago          object
facturas_impagas      int64
frecuencia_uso        int64
tickets_soporte       int64
tipo_contrato        object
cambios_plan          int64
canal_adquisicion    object
churn                 int64
dtype: object


# **5. Encoding**

## **5.1. Identificación automática: Separamos variables por tipo de dato (numéricas vs categóricas)**

In [32]:
# Identificar variables categóricas y numéricas
numericas = df_limpio.select_dtypes(include=['int64', 'float64']).columns.tolist()
categoricas = df_limpio.select_dtypes(include=['object']).columns.tolist()

print(" ANÁLISIS DE TIPOS DE VARIABLES:")
print(f"Variables numéricas ({len(numericas)}): {numericas}")
print(f"Variables categóricas ({len(categoricas)}): {categoricas}")

 ANÁLISIS DE TIPOS DE VARIABLES:
Variables numéricas (6): ['antiguedad', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'cambios_plan', 'churn']
Variables categóricas (4): ['plan', 'metodo_pago', 'tipo_contrato', 'canal_adquisicion']


## **5.2. Análisis de categorías: Verifico cuántas categorías únicas tiene cada variable categórica**

In [33]:
# Analizar categorías únicas en variables categóricas
if categoricas:
    print("\n ANÁLISIS DE CATEGORÍAS:")
    for col in categoricas:
        unique_vals = df_limpio[col].nunique()
        categories = df_limpio[col].unique()
        print(f"{col}: {unique_vals} categorías → {list(categories)}")
        print('-' * 50)


 ANÁLISIS DE CATEGORÍAS:
plan: 3 categorías → ['estandar', 'premium', 'basico']
--------------------------------------------------
metodo_pago: 4 categorías → ['transferencia_bancaria', 'efectivo', 'tarjeta_credito', 'tarjeta_debito']
--------------------------------------------------
tipo_contrato: 2 categorías → ['mensual', 'anual']
--------------------------------------------------
canal_adquisicion: 4 categorías → ['call_center', 'referido', 'web', 'redes_sociales']
--------------------------------------------------


In [34]:
df_limpio.sample(5)

,antiguedad,plan,metodo_pago,facturas_impagas,frecuencia_uso,tickets_soporte,tipo_contrato,cambios_plan,canal_adquisicion,churn
3310,60,estandar,efectivo,2,12,6,mensual,3,call_center,1
3377,22,basico,transferencia_bancaria,1,4,9,mensual,3,web,1
2228,52,estandar,transferencia_bancaria,0,3,6,mensual,3,referido,0
2062,10,premium,transferencia_bancaria,1,22,2,mensual,3,referido,0
362,38,premium,efectivo,3,30,7,mensual,0,call_center,0


In [35]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4251 entries, 0 to 4250
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   antiguedad         4251 non-null   int64 
 1   plan               4251 non-null   object
 2   metodo_pago        4251 non-null   object
 3   facturas_impagas   4251 non-null   int64 
 4   frecuencia_uso     4251 non-null   int64 
 5   tickets_soporte    4251 non-null   int64 
 6   tipo_contrato      4251 non-null   object
 7   cambios_plan       4251 non-null   int64 
 8   canal_adquisicion  4251 non-null   object
 9   churn              4251 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 332.2+ KB


# 6. Guardo el Dataset Limpio

In [36]:
df_limpio.to_csv('dataset_limpio_final.csv', index=False)

In [37]:
df_limpio['churn'].value_counts(normalize=True)


,proportion
churn,
0,0.746177
1,0.253823
